<a href="https://colab.research.google.com/github/shivanandiyer/ANLP_G33/blob/main/pymed_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install pymed and Berttopic

https://pypi.org/project/pymed/
https://maartengr.github.io/BERTopic/index.html


In [ ]:
!pip install pymed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 56.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

# Search and retrieve Pubmed Data to a dataframe via Pymed API

In [ ]:
from pymed import PubMed
pubmed = PubMed(tool="PubMedSearcher", email="shivanand.iyer@gmail.com")

############################################################################################
## PUT YOUR SEARCH TERM HERE ##
search_term = "Cardiovascular Diseases"
## PUT YOUR SEARCH TERM HERE ##
############################################################################################

results = pubmed.query(search_term, max_results=5000)
articleList = []
articleInfo = []

for article in results:
# Print the type of object we've found (can be either PubMedBookArticle or PubMedArticle).
# We need to convert it to dictionary with available function
    articleDict = article.toDict()
    articleList.append(articleDict)

# Generate list of dict records which will hold all article details that could be fetch from PUBMED API
for article in articleList:


#Sometimes article['pubmed_id'] contains list separated with comma - take first pubmedId in that list - thats article pubmedId
    pubmedId = article['pubmed_id'].partition('\n')[0]
    # Append article info to dictionary 
    #articleInfo.append({u'pubmed_id':pubmedId,
    #                   u'title':article['title'],
    #                   u'keywords':article['keywords'],
    #                   u'journal':article['journal'],
    #                   u'abstract':article['abstract'],
    #                   u'conclusions':article['conclusions'],
    #                   u'methods':article['methods'],
    #                   u'results': article['results'],
    #                   u'copyrights':article['copyrights'],
    #                   u'doi':article['doi'],
    #                   u'publication_date':article['publication_date'], 
    #                   u'authors':article['authors']})
    articleInfo.append({u'pubmed_id':pubmedId,
                       u'title':article['title'],
                       u'abstract':article['abstract'],
                       u'publication_date':article['publication_date']})
articlesPD = pd.DataFrame.from_dict(articleInfo)
articlesPD=articlesPD.dropna()
export_csv = df.to_csv ('cardiovascular_diseases.csv', index = None, header=True) 
#Print first 10 rows of dataframe
articlesPD.head(10)

'NoneType' object has no attribute 'findall'


,pubmed_id,title,abstract
0,37120719,Perivascular localized cells commit erythropoi...,"Tumors possess incessant growth features, and ..."
1,37120688,Therapeutic efficacy of a MMAE-based anti-DR5 ...,Pancreatic cancer (PC) is among the most aggre...
2,37120685,Dietary phytochemical index is favorably assoc...,Phytochemicals are bioactive compounds found i...
3,37120663,Sex differences in the efficacy and safety of ...,Despite the reported sex-related variations in...
4,37120596,Recognition of an Ala-rich C-degron by the E3 ...,The ribosome-associated quality-control (RQC) ...
5,37120549,OSMR deficiency aggravates pressure overload-i...,Oncostatin M (OSM) is a secreted cytokine of t...
6,37120545,"Association of abdominal adiposity, hepatic sh...",Abdominal ectopic fat deposition and excess vi...
7,37120464,Heart Disease and Ageing: The Roles of Senesce...,During ageing molecular damage leads to the ac...
8,37120233,Stability and bioactivity of peptides in food ...,Bioactive peptides (BPs) generated from food p...
10,37120119,Exercise and cardiovascular health: A state-of...,Cardiovascular (CV) disease (CVD) is the leadi...


## Convert Article Abstracts to a list for Topic Modelling

In [ ]:
docs=articlesPD["abstract"].tolist()

## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. 

Calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 


In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words="english")

topic_model = BERTopic(language="english", calculate_probabilities=False, verbose=True,vectorizer_model=vectorizer_model)
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/139 [00:00<?, ?it/s]

2023-04-30 11:57:19,194 - BERTopic - Transformed documents to Embeddings
2023-04-30 11:57:31,187 - BERTopic - Reduced dimensionality
2023-04-30 11:57:31,399 - BERTopic - Clustered reduced embeddings


# Extracting Topics
After fitting the model, we look at the results- the most frequent topics first as they best represent the collection of documents

In [ ]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,-1,1363,-1_patients_study_risk_disease
1,0,142,0_cholesterol_ldlc_statin_pcsk9
2,1,131,1_genetic_variants_variant_gene
3,2,130,2_nil_safflower_cha_decade
4,3,116,3_covid19_sarscov2_infection_vaccination


## Select the most frequent topic

In [ ]:
topic_model.get_topic(0)  

[('cholesterol', 0.026457267412843214),
 ('ldlc', 0.02152494382541958),
 ('statin', 0.021411535582233136),
 ('pcsk9', 0.019062642264476643),
 ('lipoprotein', 0.01770335250443675),
 ('lipid', 0.01620428133164299),
 ('cardiovascular', 0.015238576099131002),
 ('atherosclerotic', 0.013667233818115205),
 ('ldl', 0.01335565515610385),
 ('statins', 0.013029665256459762)]

In [ ]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,"Tumors possess incessant growth features, and ...",7,7_mm_myeloma_multiple_cell,mm - myeloma - multiple - cell - cells - malig...,0.316937,False
1,Pancreatic cancer (PC) is among the most aggre...,7,7_mm_myeloma_multiple_cell,mm - myeloma - multiple - cell - cells - malig...,0.305165,False
2,Phytochemicals are bioactive compounds found i...,-1,-1_patients_study_risk_disease,patients - study - risk - disease - cardiovasc...,0.000000,False
3,Despite the reported sex-related variations in...,3,3_covid19_sarscov2_infection_vaccination,covid19 - sarscov2 - infection - vaccination -...,1.000000,False
4,The ribosome-associated quality-control (RQC) ...,-1,-1_patients_study_risk_disease,patients - study - risk - disease - cardiovasc...,0.000000,False
...,...,...,...,...,...,...
4430,,2,2_nil_safflower_cha_decade,nil - safflower - cha - decade - ca - 10 - - ...,0.766155,False
4431,Fontan-associated liver disease (FALD) is one ...,65,65_transplant_posttransplant_transplantation_r...,transplant - posttransplant - transplantation ...,1.000000,False
4432,Natural menopause is defined as the cessation ...,18,18_women_pcos_menopause_estrogen,women - pcos - menopause - estrogen - und - me...,0.822837,False
4433,Nonischemic cardiomyopathies are a frequent oc...,10,10_ventricular_arrhythmias_cmr_left,ventricular - arrhythmias - cmr - left - cardi...,0.900488,False


## Visualise Topics

### The Intertopic Distance Map represents the prevalance of the topics and how closely they are related to one another

In [ ]:
topic_model.visualize_topics()

### Topic Hierarchy

We can see that topics with the same colors have been grouped together. For instance:

Topic 29 (cardiotoxicity, cardiac, myocardial) & 40 (diabetic, cardiac) have been grouped together because of their closeness.

This info is helpful to better understand the reason why the topics have been considered to be similar to one another.

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=50)

## Topic Word Scores

Visualize the selected terms for a few topics  and their c-TF-IDF scores for each topic representation. Also comparing the topic representation to each other 

In [ ]:
topic_model.visualize_barchart(top_n_topics=10)

## Topic Similarity

Similarity matrix generated by applying cosine similarities through those topic embeddings indicates how similar certain topics are to each other.

In [ ]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

In [ ]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(docs, calculate_tokens=True)

# Visualize the token-level distributions
df = topic_model.visualize_approximate_distribution(docs[2], topic_token_distr[1])
df

100%|██████████| 5/5 [00:18<00:00,  3.79s/it]


In [ ]:
df = topic_model.visualize_approximate_distribution(docs[3], topic_token_distr[3])
df

# **Search Topics**

`find_topics` to search for topics that are similar 
to an input search_term. Searching for topics that closely relate the search term. Then, we extract the most similar topic and check the results: 

In [ ]:
similar_topics, similarity = topic_model.find_topics("covid", top_n=5); similar_topics

[3, 61, -1, 8, 70]

In [ ]:
topic_model.get_topic(50)

[('smoking', 0.0718839092243426),
 ('tobacco', 0.05621649425911),
 ('cigarette', 0.045379573574771195),
 ('smokers', 0.04517631397006534),
 ('cigarettes', 0.037773429233924946),
 ('cessation', 0.03459922371104958),
 ('nicotine', 0.03312818917751141),
 ('use', 0.025183994592083717),
 ('ecigarette', 0.02159779261350297),
 ('hwls', 0.019750027775751557)]

# Article Summarisation

In [ ]:
abs=articlesPD.head(10)
type(abs)

pandas.core.frame.DataFrame

## Summarisation using transfromer library pipeline distilbart summarisation


In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization", min_length=50, max_length=150)
abs['summary'] = abs['abstract'].apply(summarizer)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Your max_length is set to 150, but you input_length is only 144. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=72)


In [ ]:
abs

,pubmed_id,title,abstract,summary
0,37120719,Perivascular localized cells commit erythropoi...,"Tumors possess incessant growth features, and ...",[{'summary_text': ' Tumors possess incessant g...
1,37120688,Therapeutic efficacy of a MMAE-based anti-DR5 ...,Pancreatic cancer (PC) is among the most aggre...,[{'summary_text': ' Pancreatic cancer is among...
2,37120685,Dietary phytochemical index is favorably assoc...,Phytochemicals are bioactive compounds found i...,[{'summary_text': ' Phytochemicals are bioacti...
3,37120663,Sex differences in the efficacy and safety of ...,Despite the reported sex-related variations in...,[{'summary_text': ' A total of 3259 LTCF resid...
4,37120596,Recognition of an Ala-rich C-degron by the E3 ...,The ribosome-associated quality-control (RQC) ...,"[{'summary_text': ' In mammals, the E3 ligase ..."
5,37120549,OSMR deficiency aggravates pressure overload-i...,Oncostatin M (OSM) is a secreted cytokine of t...,[{'summary_text': ' Oncostatin M (OSM) is a se...
6,37120545,"Association of abdominal adiposity, hepatic sh...",Abdominal ectopic fat deposition and excess vi...,[{'summary_text': ' Abdominal ectopic fat depo...
7,37120464,Heart Disease and Ageing: The Roles of Senesce...,During ageing molecular damage leads to the ac...,[{'summary_text': ' During ageing molecular da...
8,37120233,Stability and bioactivity of peptides in food ...,Bioactive peptides (BPs) generated from food p...,[{'summary_text': ' Bioactive peptides (BPs) g...
10,37120119,Exercise and cardiovascular health: A state-of...,Cardiovascular (CV) disease (CVD) is the leadi...,[{'summary_text': ' Cardiovascular (CV) diseas...
